In [1]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("gamer")

<Experiment: artifact_location='mlflow-artifacts:/431548450445816159', creation_time=1718785153935, experiment_id='431548450445816159', last_update_time=1718785153935, lifecycle_stage='active', name='gamer', tags={}>

In [10]:
import subprocess
import mlflow
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

train_data_path = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
valid_data_path = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"

# Load the data
df_train = pd.read_parquet(train_data_path)
df_valid = pd.read_parquet(valid_data_path)

# Feature engineering (example features)
df_train['duration'] = (df_train['tpep_dropoff_datetime'] - df_train['tpep_pickup_datetime']).dt.total_seconds() / 60
df_valid['duration'] = (df_valid['tpep_dropoff_datetime'] - df_valid['tpep_pickup_datetime']).dt.total_seconds() / 60

# Drop rows with duration outliers
df_train = df_train[(df_train['duration'] >= 1) & (df_train['duration'] <= 60)]
df_valid = df_valid[(df_valid['duration'] >= 1) & (df_valid['duration'] <= 60)]

# Example target and feature columns
target_col = 'duration'
feature_cols = ['passenger_count', 'trip_distance']

# Select the features and target
X_train = df_train[feature_cols]
y_train = df_train[target_col]
X_val = df_valid[feature_cols]
y_val = df_valid[target_col]

# Handle missing values by filling them with the mean of the column
X_train = X_train.fillna(X_train.mean())
X_val = X_val.fillna(X_val.mean())

alpha = 0.1

# Run MLflow experiment
with mlflow.start_run():
    mlflow.set_tag("developer", "sandeep")
    mlflow.log_param("train-data-path", train_data_path)
    mlflow.log_param("valid-data-path", valid_data_path)
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

# Output message
print("MLflow run completed. RMSE:", rmse)

MLflow run completed. RMSE: 10.063943913160992


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [3]:
import xgboost as xgb

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [5]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

mlflow.xgboost.autolog(disable=True)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [05:16:37] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.08333                           
[1]	validation-rmse:5.37773                           
[2]	validation-rmse:5.26929                           
[3]	validation-rmse:5.24950                           
[4]	validation-rmse:5.24479                           
[5]	validation-rmse:5.24343                           
[6]	validation-rmse:5.24299                           
[7]	validation-rmse:5.24285                           
[8]	validation-rmse:5.24280                           
[9]	validation-rmse:5.24278                           
[10]	validation-rmse:5.24278                          
[11]	validation-rmse:5.24278                          
[12]	validation-rmse:5.24277                          
[13]	validation-rmse:5.24277                          
[14]	validation-rmse:5.24277                          
[15]	validation-rmse:5.24277                          
[16]	validation-rmse:5.24277                          
[17]	validation-rmse:5.24277                          
[18]	valid

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [05:16:55] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:5.28210                                                    
[1]	validation-rmse:5.24394                                                    
[2]	validation-rmse:5.24283                                                    
[3]	validation-rmse:5.24278                                                    
[4]	validation-rmse:5.24278                                                    
[5]	validation-rmse:5.24277                                                    
[6]	validation-rmse:5.24277                                                    
[7]	validation-rmse:5.24277                                                    
[8]	validation-rmse:5.24277                                                    
[9]	validation-rmse:5.24277                                                    
[10]	validation-rmse:5.24277                                                   
[11]	validation-rmse:5.24277                                                   
[12]	validation-rmse:5.24277            

...

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [05:53:02] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.27411                                                     
[1]	validation-rmse:8.59597                                                     
[2]	validation-rmse:8.01986                                                     
[3]	validation-rmse:7.53341                                                     
[4]	validation-rmse:7.12521                                                     
[5]	validation-rmse:6.78477                                                     
[6]	validation-rmse:6.50249                                                     
[7]	validation-rmse:6.26969                                                     
[8]	validation-rmse:6.07864                                                     
[9]	validation-rmse:5.92248                                                     
[10]	validation-rmse:5.79530                                                    
[11]	validation-rmse:5.69199                                                    
[12]	validation-rmse:5.60824

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [05:53:47] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.62717                                                     
[1]	validation-rmse:9.22168                                                     
[2]	validation-rmse:8.84922                                                     
[3]	validation-rmse:8.50734                                                     
[4]	validation-rmse:8.19435                                                     
[5]	validation-rmse:7.90806                                                     
[6]	validation-rmse:7.64692                                                     
[7]	validation-rmse:7.40897                                                     
[8]	validation-rmse:7.19250                                                     
[9]	validation-rmse:6.99596                                                     
[10]	validation-rmse:6.81796                                                    
[11]	validation-rmse:6.65681                                                    
[12]	validation-rmse:6.51134

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [6]:
mlflow.xgboost.autolog(disable=True)

In [11]:
import pickle
from sklearn.feature_extraction import DictVectorizer


dv = DictVectorizer()
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.1212222356681564,
        'max_depth': 4,
        'min_child_weight': 3.830060509351922,
        'objective': 'reg:linear',
        'reg_alpha': 0.17455067131841767,
        'reg_lambda': 0.011875948364425093,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [06:40:31] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.22726
[1]	validation-rmse:8.51717
[2]	validation-rmse:7.92274
[3]	validation-rmse:7.42540
[4]	validation-rmse:7.01290
[5]	validation-rmse:6.67443
[6]	validation-rmse:6.39731
[7]	validation-rmse:6.17158
[8]	validation-rmse:5.98982
[9]	validation-rmse:5.84320
[10]	validation-rmse:5.72645
[11]	validation-rmse:5.63233
[12]	validation-rmse:5.55675
[13]	validation-rmse:5.49680
[14]	validation-rmse:5.44888
[15]	validation-rmse:5.41045
[16]	validation-rmse:5.37971
[17]	validation-rmse:5.35523
[18]	validation-rmse:5.33593
[19]	validation-rmse:5.32005
[20]	validation-rmse:5.30717
[21]	validation-rmse:5.29697
[22]	validation-rmse:5.28839
[23]	validation-rmse:5.28151
[24]	validation-rmse:5.27581
[25]	validation-rmse:5.27126
[26]	validation-rmse:5.26755
[27]	validation-rmse:5.26425
[28]	validation-rmse:5.26088
[29]	validation-rmse:5.25880
[30]	validation-rmse:5.25647
[31]	validation-rmse:5.25494
[32]	validation-rmse:5.25314
[33]	validation-rmse:5.25209
[34]	validation-rmse:5.2

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [06:41:13] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/home/codespace/.python/current/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way 

In [13]:
import mlflow
logged_model = 'runs:/d70b1b5572c942b59fcdd359ea982283/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)



/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [06:43:46] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [14]:
xg_boost=mlflow.xgboost.load_model(logged_model)


/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [06:47:05] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [15]:
y_pred=xg_boost.predict(valid)

In [16]:
y_pred[:10]

array([ 3.2680697, 41.28145  , 17.959467 , 21.79309  , 24.228891 ,
        8.87723  , 23.409483 ,  4.1932244,  5.8923035, 10.545562 ],
      dtype=float32)